first let's test things!

In [80]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = [6, 6]
%matplotlib widget
from PIL import Image
from tensorflow import keras

In [2]:
# load the satelite images as well as the targets: the ice concentrations and types ('stage')
# separate folder for each month's data
rgb={}
conc={}
stage={}
for i in range(1,13):
    rgb[i]=np.asarray(Image.open(str(i)+'/rgb.png'))[:,:,:3] # don't need the transparency value of the png
    stage[i]=np.asarray(Image.open(str(i)+'/stage.png'))[:,:,:3]
    conc[i]=np.asarray(Image.open(str(i)+'/conc.png'))[:,:,:3]

In [3]:
plt.figure()
plt.imshow(rgb[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
plt.figure()
plt.imshow(conc[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
indices = np.moveaxis(np.indices(rgb[1].shape)[0:2,:,:,0], 0, 2)
allArray = np.dstack((indices, rgb[1])).reshape((-1, 5))
targetarray = np.dstack((indices, conc[1])).reshape((-1, 5))
data = pd.DataFrame(allArray, columns=['y', 'x', 'red','green','blue'])
targetDF = pd.DataFrame(targetarray, columns=['y', 'x', 'red','green','blue'])

In [24]:
print(data.shape)
print(targetDF.shape)

(8697360, 5)
(8697360, 5)


In [37]:
forbiddencolors=np.array(((135,135,135),(79,255,99),(255,255,255),(0,0,0)))
colorlabels=['cyan','green','yellow','offyellow','orange','red','white','gray']
colors=np.array(((0,233,233),(57,163,73),(255,251,7),(252,215,0),(255,128,1),(227,26,28),(255,255,255),(135,135,135)))

In [47]:
datamatch=pd.Series(np.ones(len(data), dtype=bool))
targetcolormatch=pd.Series(np.zeros(len(data), dtype=bool))
for i in range(0,len(forbiddencolors)):
    datamatch=datamatch&(~((data.loc[:,'red']==forbiddencolors[i,0])&(data.loc[:,'green']==forbiddencolors[i,1])&(data.loc[:,'blue']==forbiddencolors[i,2])))
for i in range(0,len(colors)):
    targetcolormatch=targetcolormatch|((targetDF.loc[:,'red']==colors[i,0])&(targetDF.loc[:,'green']==colors[i,1])&(targetDF.loc[:,'blue']==colors[i,2]))
fullmatch=datamatch & targetcolormatch
cleandata=data.loc[fullmatch,:]
cleantargets=targetDF.loc[fullmatch,:]

In [48]:
print(cleandata.shape)
print(cleantargets.shape)

(1943723, 5)
(1943723, 5)


In [49]:
#repopulate the picture with only the data shown
newpic=np.zeros(rgb[1].shape)
newpic[cleandata.y,cleandata.x,0]=cleandata.red
newpic[cleandata.y,cleandata.x,1]=cleandata.green
newpic[cleandata.y,cleandata.x,2]=cleandata.blue

In [50]:
plt.figure()
plt.imshow(newpic.astype(int))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
target=pd.DataFrame({'y':cleantargets.y,'x':cleantargets.x,'target':np.zeros(len(cleandata))})
for i in range(0,len(colors)):
    target.loc[(cleantargets.loc[:,'red']==colors[i,0])&(cleantargets.loc[:,'green']==colors[i,1])&(cleantargets.loc[:,'blue']==colors[i,2]),'target']=i
target.loc[target.target==3]=2

In [90]:
target.target.value_counts()

5.0    938720
6.0    909350
0.0     43960
1.0     23864
2.0     11675
4.0     10792
7.0      5362
Name: target, dtype: int64

In [103]:
x_train_all, x_validate, y_train_all, y_validate = model_selection.train_test_split(cleandata.loc[:,'red':'blue'], target.target, test_size=0.1)

# now split the training data further into training and testing
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train_all, y_train_all, test_size=0.2)

In [106]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
modelGNB=gnb.fit(x_train, y_train)
y_pred = modelGNB.predict(x_test)

In [107]:
score = accuracy_score(y_test, y_pred)
print(score)

0.8801612027324435


In [108]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')

In [126]:
from sklearn.ensemble import RandomForestClassifier
RFclf=RandomForestClassifier(max_depth=10,n_estimators=600)
datafilter=np.random.choice(a=[False, True], size= len(y_train), p=[0.98, 0.02])
modelRF=RFclf.fit(x_train.loc[datafilter,:],y_train.loc[datafilter])

In [128]:
y_pred = modelRF.predict(x_test)

In [129]:
score = accuracy_score(y_test, y_pred)
print(score)

0.9417869494383628


In [130]:
plt.figure()
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, annot=True, fmt='d', cbar=True) 
plt.ylabel('true label')
plt.xlabel('predicted label')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 36.72222222222221, 'predicted label')